In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression as lr

**Importing DataSets**

In [36]:
fake = pd.read_csv("Fake (1).csv")
true = pd.read_csv("True (1).csv")

In [37]:
fake['class'] = 0
true['class'] = 1

In [38]:
print(fake.shape,true.shape)

(23481, 5) (21417, 5)


In [39]:
df_merge = pd.concat([fake,true],axis=0)
df_merge.shape

(44898, 5)

In [40]:
print(df_merge.columns)
df = df_merge.drop(['title','subject','date'],axis=1)

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')


**Sampling** 

In [41]:
df = df.sample(frac=1)

In [42]:
df.reset_index(inplace = True)
df.drop(['index'],axis=1,inplace = True)
df.head()

,text,class
0,I think this is hope over reality longtime De...,0
1,Speaking at the National Council of La Raza co...,0
2,"Trump made a bad decision, or so his ardent fo...",0
3,"Like Megyn Kelly, who lost her supporters when...",0
4,Many people have raised the alarm regarding th...,0


**Missing Values**

In [43]:
df.isna().sum()

text     0
class    0
dtype: int64

**Text Preprocessing**

* converting words into lowercase

In [44]:
df['text'] = df['text'].str.lower()

* removing leading and trailing whitespaces

In [45]:
df['text'] = df['text'].str.strip()

* removing puctuations

In [46]:
df['text'] = df['text'].str.replace('[^a-zA-Z]',' ')

C:\Users\Shams\AppData\Local\Temp/ipykernel_14328/3800430821.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^a-zA-Z]',' ')


* expanding contractions

**Defining dependent and independent variables**

In [47]:
X = df['text']
y = df['class']

**Splitting Training and Testing**

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25,random_state =42)

**Convert text to vectors** 
* CountVectorizer

In [49]:
cv = CountVectorizer(stop_words='english')
count_train= cv.fit_transform(X_train)
count_test = cv.transform(X_test)
print(cv.get_feature_names_out()[:10])

['aa' 'aaa' 'aaaaaaaand' 'aaaaackkk' 'aaaaapkfhk' 'aaaahhhh' 'aaaand'
 'aaaarrgh' 'aab' 'aabfsv']


**Convert text to vectors** 
* TfidfVectorizer

In [50]:
tv = TfidfVectorizer(stop_words='english',max_df = 0.9)
tfidf_train= tv.fit_transform(X_train)
tfidf_test = tv.transform(X_test)
print(tv.get_feature_names_out()[:10])

['aa' 'aaa' 'aaaaaaaand' 'aaaaackkk' 'aaaaapkfhk' 'aaaahhhh' 'aaaand'
 'aaaarrgh' 'aab' 'aabfsv']


**Create the CountVectorizer & TfidfVectorizer DataFrame**

In [51]:
#cv_df = pd.DataFrame(count_train.toarray(),columns = cv.get_feature_names_out())
#tfidf_df = pd.DataFrame(tfidf_train.toarray(),columns = tv.get_feature_names_out())

In [52]:
#print(count_train.head(1))

**Multinomial Naive Bayes classifier**

In [64]:
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train,y_train)
y_pred = nb_classifier.predict(count_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[5592  298]
 [ 237 5098]]
0.9523385300668151


LogisticRegression

In [66]:
lr_classifier  = lr()
lr_classifier.fit(count_train,y_train)
y_pred = lr_classifier.predict(count_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[5865   25]
 [  24 5311]]
0.9956347438752784


C:\Users\Shams\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
